# Tetris player

an RL model that can play Tetris game from (https://play.tetris.com/)

In [2]:
# !uv add pandas numpy matplotlib seaborn scikit-learn jupyter
# !uv add torch torchvision torchmetrics
# !uv add flask flask-cors

## Client-Side: data send

### Tampermonkey

```javascript
// ==UserScript==
// @name         Remote Executor (HTTP Polling)
// @match        https://play.tetris.com/
// @grant        GM_xmlhttpRequest
// @connect      192.168.0.156
// ==/UserScript==

(function () {
    const SERVER = 'http://192.168.0.156:8766';

    const poll = () => {
        GM_xmlhttpRequest({
            method: 'GET',
            url: SERVER + '/command',
            onload: (response) => {
                if (response.status === 200 && response.responseText.trim()) {
                    const code = response.responseText;

                    // Wrap user code in an async function
                    let result;
                    try {
                        // Use new Function to avoid eval restrictions and enable async
                        const fn = new Function('return (async () => {' + code + '})()');
                        result = fn(); // This returns a Promise
                    } catch (e) {
                        // Syntax error or function creation failed
                        sendResult({ error: '[JS Parse Error] ' + (e.stack || e.toString()) });
                        return;
                    }

                    // Handle Promise resolution
                    Promise.resolve(result)
                        .then(value => {
                            // Serialize result safely
                            try {
                                sendResult({ result: value });
                            } catch (e) {
                                sendResult({ error: '[Serialize Error] ' + (e.stack || e.toString()) });
                            }
                        })
                        .catch(err => {
                            sendResult({ error: String(err) });
                        });
                }
            },
            onerror: (err) => {
                console.error('[RemoteExec] Poll error:', err);
            },
            timeout: 5000
        });
    };

    function sendResult(payload) {
        GM_xmlhttpRequest({
            method: 'POST',
            url: SERVER + '/result',
            data: JSON.stringify(payload),
            headers: { "Content-Type": "application/json" }
        });
    }

    setInterval(poll, 1000); // Faster polling for responsiveness
})();
```

## Server

In [1]:
import sys
from uuid import uuid5

import websockets
import json
import time
import threading
from collections import defaultdict
import asyncio
import uuid

class WebSocketServer:
    def __init__(self, host="localhost", port=8766):
        self.host = host
        self.port = port
        self.clients = {}  # {uid: websocket}
        self.client_uid_counter = 0
        self.client_last_seen = {}  # {uid: timestamp}
        self.TIMEOUT = 45  # seconds
        self.msg_callbacks = {}  # {msg_id: callback}
        self.loop = None

    async def handle_client(self, websocket):
        # Assign unique ID
        self.client_uid_counter += 1
        client_uid = self.client_uid_counter
        self.clients[client_uid] = websocket
        self.client_last_seen[client_uid] = time.time()
        print(f"Client {client_uid} connected", flush=True, file=sys.stdout)

        try:
            async for message in websocket:
                try:
                    print(f"From {client_uid}: {message}", flush=True)
                    data = json.loads(message)

                    # validation
                    """
                    {
                      type: "ping" | "pong" | "execute" | "execution_result"
                      uid: str | int
                      timestamp: number (required only for 'ping'/'pong' messages)
                      result: any (required only for 'execution_result' messages | if no error)
                      error: any (required only for 'execution_result' messages | only if error occurs)
                      code: str (required only for 'execute' messages)
                    }
                    """
                    assert isinstance(data, dict), "Data must be a dictionary"
                    assert 'type' in data, "Data must contain 'type' field"
                    assert data['type'] in ['ping', 'pong', 'execute',
                                            'execution_result'], f"Invalid type: {data['type']}"
                    assert 'uid' in data, "Data must contain 'uid' field"
                    assert isinstance(data['uid'], (str, int)), "UID must be string or int"
                    if data['type'] in ['ping', 'pong']:
                        assert 'timestamp' in data, f"Timestamp required for {data['type']}"
                        assert isinstance(data['timestamp'], (int, float)), "Timestamp must be numeric"
                    if data['type'] == 'execute':
                        assert 'code' in data, "Code required for execute"
                        assert isinstance(data['code'], str), "Code must be string"
                    if data['type'] == 'execution_result':
                        assert 'result' in data, "Result required for execution_result"
                        assert 'error' in data, "Error required for execution_result"

                    await self.handle_message(data, client_uid)
                except json.JSONDecodeError:
                    print(f"Invalid JSON from client {client_uid}")
                except AssertionError as e:
                    print(f"Warning: Invalid message format from client {client_uid}: {e}")
        except websockets.exceptions.ConnectionClosed:
            print(f"Client {client_uid} disconnected")
        finally:
            if client_uid in self.clients:
                del self.clients[client_uid]
                if client_uid in self.client_last_seen:
                    del self.client_last_seen[client_uid]

    async def handle_message(self, data, client_uid):
        msg_type = data.get('type')
        msg_id = data.get('uid')

        # Update last seen time
        self.client_last_seen[client_uid] = time.time()

        if msg_type == 'ping':
            await self.send_message(data={'uid': msg_id, 'type': 'pong', 'timestamp': time.time()}, client_uid=client_uid)

        elif msg_type == 'execution_result':
            if msg_id in self.msg_callbacks:
                callback = self.msg_callbacks.pop(msg_id)
                asyncio.create_task(callback(data))

        else:
            print("Warn: msg has no callback!")

    async def send_message(self, data, callback=None, client_uid=None):
        client_uids = [client_uid] if client_uid is not None else self.clients.keys()

        for client_uid in client_uids:
            if client_uid not in self.clients:
                continue

            try:
                # Add message ID if not present
                if 'uid' not in data:
                    data['uid'] = str(uuid.uuid4())

                # Register callback if provided
                if callback is not None:
                    self.msg_callbacks[data['uid']] = callback

                await self.clients[client_uid].send(json.dumps(data))
                print(f"Sent message to client {client_uid}: {data}")  # Log sent message
            except websockets.exceptions.ConnectionClosed:
                # Clean up dead connection
                del self.clients[client_uid]
                if client_uid in self.client_last_seen:
                    del self.client_last_seen[client_uid]

    async def __cleanup_dead_connections(self):
        while True:
            current_time = time.time()
            dead_clients = [
                client_uid for client_uid, last_seen in self.client_last_seen.items()
                if current_time - last_seen > self.TIMEOUT
            ]

            for client_uid in dead_clients:
                print(f"Client {client_uid} timed out")
                if client_uid in self.clients:
                    await self.clients[client_uid].close()
                    del self.clients[client_uid]
                if client_uid in self.client_last_seen:
                    del self.client_last_seen[client_uid]

            await asyncio.sleep(5)  # Check every 5 seconds

    def start_server(self):
        async def run_server():
            server = await websockets.serve(self.handle_client, self.host, self.port)
            print(f"WebSocket server started on ws://{self.host}:{self.port}")

            # Start cleanup task
            cleanup_task = asyncio.create_task(self.__cleanup_dead_connections())

            await server.wait_closed()

        # Create a new event loop for this thread
        self.loop = asyncio.new_event_loop()
        asyncio.set_event_loop(self.loop)
        self.loop.run_until_complete(run_server())



    async def execute_js(self, code, timeout=0.01):
        # Generate a unique message ID for this execution
        msg_id = str(uuid.uuid4())

        # Create a future to wait for the response
        future = asyncio.Future()

        # Define the callback that will set the result of the future
        async def callback(data):
            if not future.done():
                future.set_result(data)

        # Send the execute message with the callback
        await self.send_message(
            data={
                "type": "execute",
                "code": code,
                "uid": msg_id
            },
            callback=callback
        )

        # Wait for the callback to be called and return the result
        result = await asyncio.wait_for(future, timeout=timeout)
        return result

In [2]:
server = WebSocketServer()

server_thread = threading.Thread(target=lambda: server.start_server(), daemon=True)
server_thread.start()

time.sleep(3) # just wait, to let the client connect

WebSocket server started on ws://localhost:8766
Client 1 connected


In [11]:
await server.execute_js("return 2 + 5")

Sent message to client 1: {'type': 'execute', 'code': 'return 2 + 5', 'uid': 'caf0a705-d757-45b9-9224-46f8860c93de'}
From 1: {"type":"execution_result","uid":"caf0a705-d757-45b9-9224-46f8860c93de","result":7,"error":null}


{'type': 'execution_result',
 'uid': 'caf0a705-d757-45b9-9224-46f8860c93de',
 'result': 7,
 'error': None}

- [ ] Take canvas to image (png maybe?)
- [ ] Continuesly take canvas img -> capture when images changes
- [ ] then, send the changed image via websocket
    - [ ] implement on client-side
    - [ ] implement on server (just print the uniq images)
- [ ] extract info from the images
    - [ ] level
    - [ ] score
    - [ ] lines
    - [ ] next 3 pieces
    - [ ] held piece
    - [ ] ground pieces (a matrix representing all filled cells without the current piece)
    - [ ] current piece
    - [ ]
- [ ] plug the data into rl model
- [ ] tinker with it
- [ ]